# 05: Delaunay Triangulation

*Authors: Felix Espey*

This notebook serves as supplementary learning material for the course **Geometric Algorithms**.
It showcases and explains implementations of algorithms presented in the corresponding lecture, and elaborates on some practical considerations concerning their use.
Furthermore, it offers interactive visualisations and animations.

In [ ]:
from modules.visualisation import VisualisationTool, PointSetInstance, PointsMode

vis = VisualisationTool(400,400, PointSetInstance())
vis.display()

TypeError: PointSetInstance.size() missing 1 required positional argument: 'self'

## Table of Contents

1. Introduction
2. Concepts
   1. Empty Circle Property
   2. Edge Flips
3. Randomised Incremental Construction
4. Voronoi diagrams
5. References

## 1. Introduction

This chapter focuses mainly on the **Delaunay Triangulation**, but also contains a small section on the connected topic **Voronoi diagrams**.

We start by laying out some concepts from the lecture that are important to the Delaunay Triangulation and then go over the algorithm to construct the Delaunay Triangulation from the lecture.

## 2. Concepts

Before we can look at the algorithm, we need to go over some basic concepts needed to define what a Delaunay Triangulation is.

## 1. Empty Circle Property

The most important concept for the Delaunay Triangulation is the **Empty Circle Property**. First, lets define some basic terms:
- a circle is defined by two parts: the interior and the boundary
- a circle is empty if no points are within its interior
- a circle C<sub>p,q,r</sub> is the the circle whose boundary goes through the points p, q and r
  - since p,q and r are on the boundary, the circle is still empty

The empty Circle Property states that
1. given a Delaunay Triangulation, the circumcircle C<sub>p,q,r</sub> of all triangles pqr is empty
2. a Triangulation in which the circumcircles C<sub>p,q,r</sub> of all triangles are empty is a Delaunay Triangulation

# 2. Edge Flips

Edge flips are used to increase the size of the smallest angle between two triangles by flipping the shared edge of those triangles.
In the image below TODO, the four points *p,r,q,s* form the two triangles *pqr* and *pqs* with the shared edge *pq*.
By flipping the edge *pq* to *sr*, the initial triangles are destroyed and two new triangles *prs* and *qrs* are formed, which have a larger smallest angle. If flipping an edge leads to an increase of the minimal angle of the triangulation, the edge is called illegal. A triangulation with no illegal edges is called legal.

The image below shows an example of an edge flip increasing the minimal angle in a triangulation.


<img style='float: left;' src='./images/06-image01_placeholder.PNG'>

Another way to find illegal edges is by calculating the cirumcircle C<sub>*pqr*</sub> of *pqr* and checking if the point *s* is containined in the circumcircle. The code below creates a drawing in which all illegal edges of a triangulation can be highlighted. To make sure the input to our method is always a triangulation, we add 3 dummy vertices far away which can be discarded later. Adding a point into a face also adds edges to all corners of the face, ensuring the triangulation is not destroyed. The animation also highlights the circumcircle and opposing vertex.

In [1]:
from modules.visualisation import VisualisationTool, TriangleInstance, TriangleMode, IllegalEdgeMode
from modules.data_structures import Triangulation
from modules.data_structures.animation_objects import EdgeAnimator
from modules.geometry import Point


def highlight_illegal_edges(t : Triangulation) -> EdgeAnimator:
    animator = EdgeAnimator()
    for e in t.edges:
        #skip all outer edges as they have no second triangle and therefore can not be illegal
        if e.incident_face is t.outer_face or e.twin.incident_face is t.outer_face:
            continue
        #skip edge if its twin was already checked 
        if animator.edge_or_twin_checked(e):
            continue
        animator.highlight_edge(e)
        animator.animate_circumcircle(e)
        if not t.is_legal(e):
            animator.add_illegal_edge(e)
        else:
            #only remove the edge highlight if the edge was not illegal
            animator.unhighlight_edge()
        animator.add_checked_edge(e)
    return animator


triangle_instance = TriangleInstance()
triangle_instance._default_number_of_random_points = 20
vis1 = VisualisationTool(400, 400, triangle_instance)

illegal_edge_mode = IllegalEdgeMode()
vis1.register_algorithm("highlight illegal edges", highlight_illegal_edges, illegal_edge_mode)
vis1.display()

Next up we will take a look at the Randomized Incremental Construction of a Triangulation from the lectur. As a reminder, the algorithm takes a set of points as an input and adds them to the Triangulation one by one in a random order. After each insertion edges are checked to ensure the Triangulation stays legal.

As before, the Triangulation starts with 3 dummy vertices far away which can be removed after the last point has been inserted.

In [2]:
from modules.data_structures.animation_objects import Incremental_Construction_Animator as RICA
from modules.visualisation import PointSetInstance
import random

triangle_mode = TriangleMode()

def randomized_incremental_construction(points : set[Point]):
    animator = RICA()
    triangle_mode.set_instance(animator._triangulation)
    random.shuffle(points)
    for p in points:
        animator.insert_point(p)

    return animator

point_instance = PointSetInstance()
point_instance._default_number_of_random_points = 20

vis2 = VisualisationTool(400, 400, point_instance)
#vis2.register_algorithm("Randomzied Incremental Construction", randomized_incremental_construction, triangle_mode)
vis2.display()

# 3. Randomized Incremental Construction

To construct the Delaunay Triangulation for a set of points we will use a randomized algorithm by inserting the points one by one. After each insertion the triangulation is leagilized by flipping all illegal edges. 

# 4. Voronoi Diagramms